In [1]:
%pylab inline
import csv
import operator
import re
from os import system
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [2]:
h5_filename ='/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/04/06/1523035800.h5'
%set_env h5_filename = $h5_filename
ants='m000,m001,m002,m003,m005,m006,m008,m010,m011,m012,m013,m014,m015,m016,m017,m020,m021,m023,m024,m025,m026,\
m027,m029,m030,m031,m032,m033,m034,m036,m039,m040,m041,m042,m043,m044,m045,m046,m047,m048,m049,m050,m051,m052,\
m053,m054,m055,m056,m057,m058,m059,m060,m061,m062,m063'
%set_env ants=$ants
ref_ant='m005'
%set_env ref_ant=$ref_ant

env: h5_filename=/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/04/06/1523035800.h5
env: ants=m000,m001,m002,m003,m005,m006,m008,m010,m011,m012,m013,m014,m015,m016,m017,m020,m021,m023,m024,m025,m026,m027,m029,m030,m031,m032,m033,m034,m036,m039,m040,m041,m042,m043,m044,m045,m046,m047,m048,m049,m050,m051,m052,m053,m054,m055,m056,m057,m058,m059,m060,m061,m062,m063
env: ref_ant=m005


In [3]:
print "Starting H"
res_h = !python /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'H' -r $ref_ant \
  -a $ants  -x '1148-001' $h5_filename
print "Done H"
print res_h

Starting H
Done H
['', 'Loading and processing data...', '', "DEBUG: Found int_time=7.996616972 in telstate['sdp_l0_int_time']", "DEBUG: Found scale_factor_timestamp=1712000000.0 in telstate[u'i0_scale_factor_timestamp']", "DEBUG: Found sync_time=1523010668.0 in telstate[u'i0_sync_time']", "DEBUG: Found n_chans=4096 in telstate['sdp_l0_n_chans']", "DEBUG: Found bandwidth=856000000.0 in telstate['sdp_l0_bandwidth']", "DEBUG: Found center_freq=1284000000.0 in telstate['sdp_l0_center_freq']", 'antennas (54): m000, m001, m002, m003, m005 (*ref*), m006, m008, m010, m011, m012, m013, m014, m015, m016, m017, m020, m021, m023, m024, m025, m026, m027, m029, m030, m031, m032, m033, m034, m036, m039, m040, m041, m042, m043, m044, m045, m046, m047, m048, m049, m050, m051, m052, m053, m054, m055, m056, m057, m058, m059, m060, m061, m062, m063 [pol H]', 'baselines (1431): 0-1 0-2 1-2 0-3 1-3 2-3 0-4 1-4 2-4 3-4 0-5 1-5 2-5 3-5 4-5 0-6 1-6 2-6 3-6 4-6 5-6 0-7 1-7 2-7 3-7 4-7 5-7 6-7 0-8 1-8 2-8 3-8 4

In [4]:
print "Starting V"
res_v = !python /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'V' -r $ref_ant \
  -a $ants -x '1148-001' $h5_filename
print "Done V"

Starting V
Done V


In [5]:
delays_h = {}
for i,r in enumerate(res_h):
    if r.startswith('Antenna'):
        ant = r[8:12]
        delays_h[ant] = {}
        delays_h[ant]['E'] = float(res_h[i+2].split(':')[1].lstrip().split(' ')[0])
        delays_h[ant]['N'] = float(res_h[i+3].split(':')[1].lstrip().split(' ')[0])
        delays_h[ant]['U'] = float(res_h[i+4].split(':')[1].lstrip().split(' ')[0])
        delays_h[ant]['D'] = float(res_h[i+5].split(':')[1].lstrip().split(' ')[0])
delays_h

{'m000': {'D': 5875.797, 'E': -8.261, 'N': -207.288, 'U': 1.202},
 'm001': {'D': 5869.959, 'E': 1.16, 'N': -171.773, 'U': 1.107},
 'm002': {'D': 5871.467, 'E': -32.096, 'N': -224.24, 'U': 1.264},
 'm003': {'D': 5872.778, 'E': -66.496, 'N': -202.283, 'U': 0.914},
 'm005': {'D': 5877.82, 'E': -102.088, 'N': -283.12, 'U': 1.475},
 'm006': {'D': 5880.204, 'E': -18.232, 'N': -295.427, 'U': 1.781},
 'm008': {'D': 5874.904, 'E': -93.505, 'N': -535.033, 'U': 3.07},
 'm010': {'D': 5881.439, 'E': 88.12, 'N': -511.88, 'U': 3.807},
 'm011': {'D': 5882.337, 'E': 84.015, 'N': -352.079, 'U': 2.751},
 'm012': {'D': 5863.331, 'E': 140.033, 'N': -368.271, 'U': 3.063},
 'm013': {'D': 5863.474, 'E': 236.835, 'N': -393.474, 'U': 3.778},
 'm014': {'D': 5868.655, 'E': 280.708, 'N': -285.803, 'U': 3.192},
 'm015': {'D': 5916.982, 'E': 210.673, 'N': -219.15, 'U': 2.372},
 'm016': {'D': 5815.612, 'E': 288.157, 'N': -185.865, 'U': 2.531},
 'm017': {'D': 5875.195, 'E': 199.669, 'N': -112.276, 'U': 1.613},
 'm020'

In [6]:
delays_v = {}
for i,r in enumerate(res_v):
    if r.startswith('Antenna'):
        ant = r[8:12]
        delays_v[ant] = {}
        delays_v[ant]['E'] = float(res_v[i+2].split(':')[1].lstrip().split(' ')[0])
        delays_v[ant]['N'] = float(res_v[i+3].split(':')[1].lstrip().split(' ')[0])
        delays_v[ant]['U'] = float(res_v[i+4].split(':')[1].lstrip().split(' ')[0])
        delays_v[ant]['D'] = float(res_v[i+5].split(':')[1].lstrip().split(' ')[0])
delays_v

{'m000': {'D': 5877.025, 'E': -8.248, 'N': -207.292, 'U': 1.219},
 'm001': {'D': 5870.967, 'E': 1.163, 'N': -171.773, 'U': 1.106},
 'm002': {'D': 5872.218, 'E': -32.092, 'N': -224.242, 'U': 1.27},
 'm003': {'D': 5874.411, 'E': -66.507, 'N': -202.279, 'U': 0.897},
 'm005': {'D': 5878.919, 'E': -102.088, 'N': -283.12, 'U': 1.475},
 'm006': {'D': 5880.995, 'E': -18.23, 'N': -295.428, 'U': 1.785},
 'm008': {'D': 5875.799, 'E': -93.524, 'N': -535.026, 'U': 3.043},
 'm010': {'D': 5882.956, 'E': 88.107, 'N': -511.875, 'U': 3.789},
 'm011': {'D': 5883.223, 'E': 83.998, 'N': -352.074, 'U': 2.726},
 'm012': {'D': 5864.349, 'E': 140.016, 'N': -368.265, 'U': 3.039},
 'm013': {'D': 5864.223, 'E': 236.82, 'N': -393.467, 'U': 3.75},
 'm014': {'D': 5870.255, 'E': 280.691, 'N': -285.796, 'U': 3.164},
 'm015': {'D': 5918.599, 'E': 210.654, 'N': -219.143, 'U': 2.343},
 'm016': {'D': 5816.706, 'E': 288.161, 'N': -185.874, 'U': 2.39},
 'm017': {'D': 5875.952, 'E': 199.632, 'N': -112.264, 'U': 1.557},
 'm02

# Cross check between polarizations for derived positions

In [7]:
for a in sorted(delays_v):
    print a,
    print("diff E %06.3f") %(delays_h[a]['E'] - delays_v[a]['E']),
    print("diff N %06.3f") %(delays_h[a]['N'] - delays_v[a]['N']),
    print("diff U %06.3f") %(delays_h[a]['U'] - delays_v[a]['U']),
    print("diff D %06.3f") %(delays_h[a]['D'] - delays_v[a]['D'])
    
# expect differences between E,N and U less than 1cm and Delays less than 3m 

m000 diff E -0.013 diff N 00.004 diff U -0.017 diff D -1.228
m001 diff E -0.003 diff N 00.000 diff U 00.001 diff D -1.008
m002 diff E -0.004 diff N 00.002 diff U -0.006 diff D -0.751
m003 diff E 00.011 diff N -0.004 diff U 00.017 diff D -1.633
m005 diff E 00.000 diff N 00.000 diff U 00.000 diff D -1.099
m006 diff E -0.002 diff N 00.001 diff U -0.004 diff D -0.791
m008 diff E 00.019 diff N -0.007 diff U 00.027 diff D -0.895
m010 diff E 00.013 diff N -0.005 diff U 00.018 diff D -1.517
m011 diff E 00.017 diff N -0.005 diff U 00.025 diff D -0.886
m012 diff E 00.017 diff N -0.006 diff U 00.024 diff D -1.018
m013 diff E 00.015 diff N -0.007 diff U 00.028 diff D -0.749
m014 diff E 00.017 diff N -0.007 diff U 00.028 diff D -1.600
m015 diff E 00.019 diff N -0.007 diff U 00.029 diff D -1.617
m016 diff E -0.004 diff N 00.009 diff U 00.141 diff D -1.094
m017 diff E 00.037 diff N -0.012 diff U 00.056 diff D -0.757
m020 diff E 00.044 diff N -0.014 diff U 00.063 diff D -1.152
m021 diff E 00.019 diff 

In [8]:
# write solutions to file
matchRe = re.findall('[0-9]{10,}', h5_filename)
filename='delay_file_'+''.join(matchRe)+'.txt'

with open(filename, 'w') as f:
    for h,v in zip(delays_h,delays_v):
        print >> f, h,v, mean([delays_h[h]['E'],delays_v[v]['E']]) , \
                    mean([delays_h[h]['N'],delays_v[v]['N']]), \
                    mean([delays_h[h]['U'],delays_v[v]['U']]), \
                    delays_h[h]['D'],delays_v[v]['D']
    f.close()

In [9]:
# sort delay file by antenna name and load antennaLOC
sample = open(filename, 'r')
sorter = csv.reader(sample, delimiter=' ')
sort = sorted(sorter, key=operator.itemgetter(1))

with open('ant_loc.csv', 'r') as f:
    tocs = f.read()

In [10]:
cols = ['Ants', 'Delay', 'East', 'North', 'UP', 'TOC']
df = pd.DataFrame(columns=cols)
for line in sort:
    p = re.compile(line[0]+',\s+\d{4}\.\d{1,3}')
    toc = float(p.search(tocs).group()[6:])
    df = df.append(pd.Series(
                             [
                                line[0],
                                line[-2],
                                line[2],
                                line[3],
                                line[4], 
                                toc
                             ], 
                              index=['Ants', 'Delay', 'East', 'North', 'UP', 'TOC']
                            ), ignore_index = True)

In [11]:
deltaTOC, deltaUP = [], []
deltaTOC.append(0)
deltaUP.append(0)
for i in range(1, len(df.TOC[1:])+1):
    deltaTOC.append(df.TOC[i] - df.TOC[i-1])
    deltaUP.append(float(df.UP[i]) - float(df.UP[i-1]))

em  = [' ' for i in range(len(deltaUP))]
df[' '] =  pd.Series(em,  index = df.index)

df['delta UP']  = pd.Series(deltaUP,  index = df.index)
df['delta TOC'] = pd.Series(deltaTOC, index = df.index)

print('\nDelay Solutions and antenna locations\n')
df


Delay Solutions and antenna locations



,Ants,Delay,East,North,UP,TOC,,delta UP,delta TOC
0,m000,5875.797,-8.2545,-207.29,1.2105,1052.205,,0.0000,0.000
1,m001,5869.959,1.1615,-171.773,1.1065,1051.874,,-0.1040,-0.331
2,m002,5871.467,-32.094,-224.241,1.267,1052.210,,0.1605,0.336
3,m003,5872.778,-66.5015,-202.281,0.9055,1051.754,,-0.3615,-0.456
4,m005,5877.82,-102.088,-283.12,1.475,1052.430,,0.5695,0.676
5,m006,5880.204,-18.231,-295.4275,1.783,1052.590,,0.3080,0.160
6,m008,5874.904,-93.5145,-535.0295,3.0565,1053.990,,1.2735,1.400
7,m010,5881.439,88.1135,-511.8775,3.798,1054.756,,0.7415,0.766
8,m011,5882.337,84.0065,-352.0765,2.7385,1053.521,,-1.0595,-1.235
9,m012,5863.331,140.0245,-368.268,3.051,1053.730,,0.3125,0.209


In [12]:
# send file to ops server to process and push to github

opsServer='ops.kat.ac.za'
rPath='/home/kat/ops_team_sw/utilities/update_delay_models'
cmd = "sshpass -p {} scp -r {} kat@{}:{}".format('kat', filename, opsServer, rPath)
system(cmd)


32512